In [1]:
from langchain_google_community import GmailToolkit

toolkit=GmailToolkit()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=639991018382-33cdj8s52nip9npccsksgj4v6lpv232b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64346%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=N9JLCrUYdJgQbXjJjZcbRIx5OQktP3&access_type=offline


In [ ]:
import os
import json
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

from langchain_google_community import GmailToolkit
from langchain_google_community.gmail.utils import build_resource_service

from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferWindowMemory
from langchain_google_genai import ChatGoogleGenerativeAI

# -------------------------------------------------------
# Gmail API Scopes & Credential Files
# -------------------------------------------------------
SCOPES = ["https://mail.google.com/"]
TOKEN_FILE = "token.json"
CREDS_FILE = "credentials.json"  # Download from Google Cloud Console

creds = None
if os.path.exists(TOKEN_FILE):
    creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CREDS_FILE, SCOPES)
        creds = flow.run_local_server(port=0)
    with open(TOKEN_FILE, "w") as token:
        token.write(creds.to_json())

# -------------------------------------------------------
# Build Gmail Toolkit
# -------------------------------------------------------
api_resource = build_resource_service(credentials=creds)
toolkit = GmailToolkit(api_resource=api_resource)
tools = toolkit.get_tools()
print("✅ Gmail Tools Loaded:", [t.name for t in tools])

# -------------------------------------------------------
# Email history functions
# -------------------------------------------------------
HISTORY_FILE = "email_history.json"

def load_history():
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, "r") as f:
            return json.load(f)
    return []

def save_history(new_entry):
    history = load_history()
    history.append(new_entry)
    history = history[-10:]
    with open(HISTORY_FILE, "w") as f:
        json.dump(history, f, indent=2)

# -------------------------------------------------------
# Memory for LLM
# -------------------------------------------------------
memory = ConversationBufferWindowMemory(k=10, memory_key="chat_history", return_messages=True)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", 
    temperature=0.2,
    instructions=(
        "The email should be professional, concise, and relevant to the job description. "
        "Write compelling, professional emails tailored to specific recruiters. "
        "Personalize content based on resume, company information and job requirements. "
        "Maintain professional tone while being engaging and authentic. "
        "Include relevant skills and experiences that match job requirements. "
        "Create clear call-to-actions. "
        "Ensure emails are concise but comprehensive. "
        "Remember previous email styles and user preferences."
        "Do use SerpAPI to search for latest job descriptions and company information."
        "Improve email quality based on previous feedback."
    ),
)

agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)

# -------------------------------------------------------
# Ask user for email details
# -------------------------------------------------------
gmail_inp = {
    "receiver": input("Receiver email: "),
    "subject": input("Subject: "),
    "message": input("Message: ")
}

save_history(gmail_inp)

# Build style history text
history_text = "\n".join(
    [f"To: {h['receiver']}, Subject: {h['subject']}, Message: {h['message']}" for h in load_history()]
)

# Prompts for Draft and Send
prompt_draft = f"""
Use the GmailDraft tool to immediately create this draft email:
Receiver: {gmail_inp['receiver']}
Subject: {gmail_inp['subject']}
Message: {gmail_inp['message']}
Match the tone and style to my recent emails:
{history_text}
Do not ask for confirmation, just draft it.
"""

prompt_send = f"""
Use the GmailSend tool to immediately send this email:
Receiver: {gmail_inp['receiver']}
Subject: {gmail_inp['subject']}
Message: {gmail_inp['message']}
Match the tone and style to my recent emails:
{history_text}
Do not ask for confirmation, just send it.
"""

# -------------------------------------------------------
# Select mode and run
# -------------------------------------------------------
select = input("prompt1/2 type (1-Draft, 2-Send): ")
if select == "1":
    print("📄 Drafting email...")
    result = agent_executor.run(prompt_draft)
else:
    print("📨 Sending email...")
    result = agent_executor.run(prompt_send)

print("✅ Result:", result)


✅ Gmail Tools Loaded: ['create_gmail_draft', 'send_gmail_message', 'search_gmail', 'get_gmail_message', 'get_gmail_thread']


C:\Users\Antriksh Sharma\AppData\Local\Temp\ipykernel_14948\2492406141.py:62: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=10, memory_key="chat_history", return_messages=True)
C:\Users\Antriksh Sharma\AppData\Local\Temp\ipykernel_14948\2492406141.py:65: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create

📨 Sending email...


> Entering new AgentExecutor chain...


C:\Users\Antriksh Sharma\AppData\Local\Temp\ipykernel_14948\2492406141.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent_executor.run(prompt_send)


Action:
```json
{
  "action": "send_gmail_message",
  "action_input": {
    "to": "akao91435@gmail.com",
    "subject": "hi",
    "message": "wtf!"
  }
}
```
Observation: Message sent. Message Id: 19895eb3dedc108c
Thought:Action:
```json
{
  "action": "Final Answer",
  "action_input": "The email has already been sent."
}
```

> Finished chain.
✅ Result: The email has already been sent.
